# Bedrock AgentとBedrock AgentCore Gatewayの統合

このノートブックでは、AWS Lambda と DynamoDB で構築された Fruit Stand バックエンド API について、Amazon Bedrock Agent と Bedrock AgentCore Gateway のエンドツーエンドの統合を示します。

## アーキテクチャの概要
```
ユーザーのクエリ → Bedrock Agent → Bridge Lambda → Bedrock AgentCore Gateway → ターゲット Lambda → DynamoDB
```

## Step 1: Install Dependencies & Setup

**日本語訳:**

## ステップ 1: 依存関係のインストールとセットアップ

まず、必要な依存関係をインストールし、プロジェクトをセットアップする必要があります。

1. 新しいターミナルウィンドウを開き、プロジェクトのルートディレクトリに移動します。

```bash
cd /path/to/project
```

2. Node.js と npm がインストールされていることを確認します。インストールされていない場合は、[公式サイト](https://nodejs.org) から最新バージョンをダウンロードしてインストールしてください。

3. 次のコマンドを実行して、プロジェクトの依存関係をインストールします。

```bash
npm install
```

4. `.env` ファイルを作成し、必要な環境変数を設定します。

```
API_KEY=your_api_key_here
```

プロジェクトのセットアップが完了したら、次のステップに進むことができます。

In [ ]:
# boto3をインストール

日本語訳:

`pip install boto3` を実行して、 boto3 をインストールしてください。 boto3 は Amazon Web Services (AWS) との対話を簡単にするための Python 用の AWS SDK です。この SDK を使うと、Python スクリプトから AWS のサービスを操作できます。

例えば、 boto3 を使えば、Amazon S3 バケットの作成、オブジェクトのアップロード/ダウンロード、Amazon EC2 インスタンスの起動/停止などの操作を Python コードで実行できます。 boto3 は AWS API をラップし、開発者が直接低レベルの AWS API を呼び出す必要がなくなります。

boto3 は Python 2.6.5 以降と Python 3.3 以降の両方で動作し、pip を使ってインストールできます。インストール後は、 `import boto3` と記述して boto3 をインポートし、AWS との対話を開始できます。
!pip3 install botocore--quiet
!pip3 install boto3 --quiet

import boto3
import json
import zipfile
import os
import urllib.request
import urllib.parse
import time

print("✅ Dependencies installed successfully")

## ステップ 2: 必要な権限を持つ IAM ロールを作成する

日本語訳:

このステップでは、Amazon EKS クラスターと連携するために必要な許可を持つ IAM ロール を作成します。この IAM ロール は、クラスターの作成時に指定されます。

1. https://console.aws.amazon.com/iam/ で AWS IAM コンソールを開きます。
2. 左側のナビゲーションペインで 「ロール」 を選択し、「ロールの作成」 をクリックします。
3. 「AWS のサービス」 を選択し、「使用事例の選択」 で 「EKS」 を選択します。
4. 「EKSクラスター」 を選択し、「次のステップ」 をクリックします。
5. 「アタッチされたアクセス許可ポリシー」 で、「AmazonEKSClusterPolicy」 と 「AmazonEKSServicePolicy」 を選択します。
6. 「次のステップ」 をクリックします。
7. 「ロール名」 を入力します (例: eksServiceRole)。
8. 「ロールの作成」 をクリックします。

作成した IAM ロール の ARN (Amazon リソースネーム) をメモしておいてください。次のステップで使用します。

In [ ]:
def create_iam_role():
    以下が日本語訳になります。

"""Bedrock Core Gateway 、 Bedrock Agent 、 Lambda で使用できる IAM ロールを作成します"""

技術的な用語である "IAM role" 、 "Bedrock Core Gateway" 、 "Bedrock Agent" 、 "Lambda" はそのまま残しました。半角英数字の前後には半角スペースを挿入しています。
    iam = boto3.client('iam')
    
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            },
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "bedrock.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            },
            {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock-agentcore.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
        ]
    }
    
    try:
        role_response = iam.create_role(
            RoleName='BedrockAgentCoreGatewayLambdaRole',
            AssumeRolePolicyDocument=json.dumps(trust_policy),
            Description='Role for Bedrock Gateway with DynamoDB access'
        )
        
        # Create and attach inline policy

インラインポリシーを作成してアタッチする

To create an inline policy for your role:

1. Open the AWS Management Console, and then open the IAM console at https://console.aws.amazon.com/iam/.
2. In the navigation pane, choose Roles, and then choose your role.
3. On the Permissions tab, expand the Inline Policies section and choose click here.
4. Choose the JSON tab.
5. Enter a JSON policy document for the policy. For example:

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "logs:CreateLogGroup",
            "Resource": "arn:aws:logs:us-west-2:123456789012:*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": [
                "arn:aws:logs:us-west-2:123456789012:log-group:/aws/lambda/my-function:*"
            ]
        }
    ]
}
```

6. When you are finished adding statements, choose Review Policy.
7. Enter a name for your new inline policy, and choose Create Policy.

The policy is now attached to your role.

ロールにインラインポリシーを作成するには:

1. AWS マネジメントコンソールを開き、https://console.aws.amazon.com/iam/ で IAM コンソールを開きます。
2. ナビゲーションペインで [Roles] を選択し、ロールを選びます。 
3. [Permissions] タブで、[Inline Policies] セクションを展開し、[click here] を選択します。
4. [JSON] タブを選択します。
5. ポリシーの JSON ドキュメントを入力します。例:

```json
{
    "Version": "2012-10-17", 
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "logs:CreateLogGroup",
            "Resource": "arn:aws:logs:us-west-2:123456789012:*"
        },
        {
            "Effect": "Allow", 
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": [
                "arn:aws:logs:us-west-2:123456789012:log-group:/aws/lambda/my-function:*"
            ]
        }
    ]
}
```

6. ステートメントの追加が完了したら、[Review Policy] を選択します。
7. 新しいインラインポリシーの名前を入力し、[Create Policy] を選択します。

ポリシーがロールにアタッチされました。
        permissions_policy = {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Action": [
                        "bedrock-agentcore:*",
                        "iam:PassRole",
                        "lambda:InvokeFunction"
                    ],
                    "Resource": "*"
                }
            ]
        }
        
        iam.put_role_policy(
            RoleName='BedrockAgentCoreGatewayLambdaRole',
            PolicyName='BedrockAgentCoreInlinePolicy',
            PolicyDocument=json.dumps(permissions_policy)
        )
        
        # Attach managed policies

日本語訳:
# 管理ポリシーを付与する

To grant permissions to a user, group, or role, you can use the AWS Management Console, AWS CLI, or AWS API operations. Managed policies are stand-alone policy documents that you can attach to multiple users, groups, and roles in your AWS account. Managed policies make it easier to re-use the same permissions across multiple entities.

ユーザー、グループ、またはロールに権限を付与するには、AWS 管理コンソール、AWS CLI、または AWS API オペレーションを使用できます。管理ポリシーは、AWS アカウント内の複数のユーザー、グループ、ロールに付与できる独立したポリシードキュメントです。管理ポリシーを使用すると、同じ権限を複数のエンティティに再利用しやすくなります。

To attach a managed policy to a user, group, or role:

ユーザー、グループ、またはロールに管理ポリシーを付与するには:

1. Sign in to the AWS Management Console and open the IAM console at https://console.aws.amazon.com/iam/.
1. AWS 管理コンソールにサインインし、https://console.aws.amazon.com/iam/ で IAM コンソールを開きます。

2. In the navigation pane, choose Users, Groups, or Roles.
2. ナビゲーションペインで、[Users]、[Groups]、または [Roles] を選択します。

3. Select the name of the user, group, or role to which you want to attach the policy.
3. ポリシーを付与するユーザー、グループ、またはロールの名前を選択します。

4. Choose the Permissions tab.
4. [Permissions] タブを選択します。

5. Under Managed Policies, choose Attach Policies.
5. [Managed Policies] の下で [Attach Policies] を選択します。

6. Select the check box next to the policy you want to attach.
6. 付与するポリシーの横にあるチェックボックスを選択します。

7. Choose Attach Policy.
7. [Attach Policy] を選択します。

You can also use the aws_iam_policy_attachment resource in Terraform to attach a managed policy to a user, group, or role. For example:

また、Terraform の aws_iam_policy_attachment リソースを使用して、ユーザー、グループ、またはロールに管理ポリシーを付与することもできます。例:

```
resource "aws_iam_user" "example" {
  name = "example_user"
}

resource "aws_iam_policy_attachment" "example" {
  name       = "example-attachment"
  users      = [aws_iam_user.example.name]
  policy_arn = "arn:aws:iam::aws:policy/ReadOnlyAccess"
}
```

This example creates an IAM user named "example_user" and attaches the "ReadOnlyAccess" managed policy to that user.

この例では、"example_user" という名前の IAM ユーザーを作成し、そのユーザーに "ReadOnlyAccess" 管理ポリシーを付与しています。
        iam.attach_role_policy(
            RoleName='BedrockAgentCoreGatewayLambdaRole',
            PolicyArn='arn:aws:iam::aws:policy/AdministratorAccess'
        )
        
        role_arn = role_response['Role']['Arn']
        print(f"✅ Created IAM role: {role_arn}")
        return role_arn
        
    except Exception as e:
        if 'already exists' in str(e):
            # Ensure existing role has all required permissions

既存の役割が必要なすべての権限を持っていることを確認します。

resource "aws_iam_role_policy_attachment" "eks_cluster_policy" {
  policy_arn = "arn:aws:iam::aws:policy/EKSClusterPolicy"
  role       = aws_iam_role.eks_cluster_role.name
}

resource "aws_iam_role_policy_attachment" "eks_service_policy" {
  policy_arn = "arn:aws:iam::aws:policy/EKSServicePolicy"
  role       = aws_iam_role.eks_cluster_role.name
}

resource "aws_iam_role_policy_attachment" "eks_cni_policy" {
  policy_arn = "arn:aws:iam::aws:policy/AmazonEKS_CNI_Policy"
  role       = aws_iam_role.eks_cluster_role.name
}

resource "aws_iam_role_policy_attachment" "ecr_readonly_policy" {
  policy_arn = "arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryReadOnly"
  role       = aws_iam_role.eks_cluster_role.name
}
            try:
                iam.attach_role_policy(
                    RoleName='BedrockAgentCoreGatewayLambdaRole',
                    PolicyArn='arn:aws:iam::aws:policy/AdministratorAccess'
                )
            except:
                pass  # ポリシーがすでに付加されている可能性があります

日本語訳:

ポリシーがすでに付加されている可能性があります。 コード、コマンド、変数名、関数名などの技術的な用語は翻訳せずにそのまま残しました。
            
            account_id = boto3.client('sts').get_caller_identity()['Account']
            role_arn = f"arn:aws:iam::{account_id}:role/BedrockAgentCoreGatewayLambdaRole"
            print(f"✅ Using existing IAM role: {role_arn}")
            return role_arn
        else:
            print(f"❌ Error creating role: {e}")
            return None

gateway_role_arn = create_iam_role()
print(f"Gateway Role ARN: {gateway_role_arn}")

## ステップ 3: DynamoDB テーブルの作成

注文と在庫追跡用の DynamoDB テーブルを作成します。

In [ ]:
def create_dynamodb_tables():
    以下が日本語訳になります。

"""注文と在庫の DynamoDB テーブルを作成する"""
    dynamodb = boto3.client('dynamodb', region_name='us-east-1')
    
    # 注文テーブルの作成

CREATE TABLE Orders (
    OrderID int NOT NULL PRIMARY KEY,
    CustomerName varchar(50) NOT NULL,
    OrderDate date NOT NULL,
    ShipperName varchar(50) NOT NULL,
    ShipperPhoneNumber varchar(20)
);

# OrderID は自動的に生成される一意の値です。
# CustomerName は注文した顧客の名前です。
# OrderDate は注文が行われた日付です。
# ShipperName は配送業者の名前です。
# ShipperPhoneNumber は配送業者の電話番号です。
    try:
        orders_table = dynamodb.create_table(
            TableName='FruitOrders',
            KeySchema=[
                {'AttributeName': 'order_id', 'KeyType': 'HASH'}
            ],
            AttributeDefinitions=[
                {'AttributeName': 'order_id', 'AttributeType': 'S'}
            ],
            BillingMode='PAY_PER_REQUEST'
        )
        print("✅ Created Orders table")
    except Exception as e:
        if 'already exists' in str(e):
            print("✅ Orders table already exists")
        else:
            print(f"❌ Error creating Orders table: {e}")
    
    # Create Inventory table

在庫テーブルを作成します

```sql
CREATE TABLE Inventory (
    id INT PRIMARY KEY,
    product_name VARCHAR(50) NOT NULL,
    quantity INT NOT NULL,
    price DECIMAL(10, 2) NOT NULL
);
```

この SQL ステートメントは、 `Inventory` という名前の新しいテーブルを作成します。このテーブルには以下の列が含まれます:

- `id`: 主キーとして機能する一意の整数値
- `product_name`: 最大 50 文字の製品名を格納する文字列
- `quantity`: 在庫数を格納する整数値
- `price`: 製品の価格を格納する 10 桁の decimal 値 (小数点以下 2 桁)

`NOT NULL` 制約により、 `product_name`、 `quantity`、 `price` 列にはデータが必須となります。
    try:
        inventory_table = dynamodb.create_table(
            TableName='FruitInventory',
            KeySchema=[
                {'AttributeName': 'fruit_name', 'KeyType': 'HASH'}
            ],
            AttributeDefinitions=[
                {'AttributeName': 'fruit_name', 'AttributeType': 'S'}
            ],
            BillingMode='PAY_PER_REQUEST'
        )
        print("✅ Created Inventory table")
        
        # 在庫データを初期化する

inventory = {
    'r' : 100,  # 赤い宝石
    'b' : 500,  # 青い宝石
    'g' : 750   # 緑の宝石
}

for gem, count in inventory.items():
    print(f'宝石 { gem } の在庫数は { count } 個です。')
        # Wait for table to be active

テーブルがアクティブになるのを待ちます。

resource_id = table.latest_stream_arns[ 'ResourceARN' ]
client = boto3.client('dynamodbstreams')

stream_response = client.describe_stream(StreamArn=resource_id)

while stream_response[ 'StreamDescription' ][ 'StreamStatus' ] != 'ENABLED':
    stream_response = client.describe_stream(StreamArn=resource_id)
    time.sleep(1)

print('Stream is enabled!')
        waiter = dynamodb.get_waiter("table_exists")
        waiter.wait(TableName="FruitInventory", WaiterConfig={"Delay": 2, "MaxAttempts": 30})
        print("✅ Table is active")
        
        
        fruits_data = [
            {'fruit_name': 'apple', 'price': 1.20, 'unit': 'each', 'stock': 100},
            {'fruit_name': 'banana', 'price': 0.50, 'unit': 'each', 'stock': 150},
            {'fruit_name': 'orange', 'price': 0.75, 'unit': 'each', 'stock': 80},
            {'fruit_name': 'strawberry', 'price': 3.99, 'unit': 'pound', 'stock': 25},
            {'fruit_name': 'blueberry', 'price': 4.50, 'unit': 'pint', 'stock': 30}
        ]
        
        for fruit in fruits_data:
            dynamodb.put_item(
                TableName='FruitInventory',
                Item={
                    'fruit_name': {'S': fruit['fruit_name']},
                    'price': {'N': str(fruit['price'])},
                    'unit': {'S': fruit['unit']},
                    'stock': {'N': str(fruit['stock'])}
                }
            )
        print("✅ Initialized inventory data")
        
    except Exception as e:
        if 'already exists' in str(e):
            print("✅ Inventory table already exists")
        else:
            print(f"❌ Error creating Inventory table: {e}")

# DynamoDBテーブルを作成する

日本語訳:

以下のコマンドを使用して、 `users` テーブルと `orders` テーブルを作成します。

```
aws dynamodb create-table \
    --table-name users \
    --attribute-definitions \
        AttributeName=user_id,AttributeType=S \
    --key-schema \
        AttributeName=user_id,KeyType=HASH \
    --provisioned-throughput \
        ReadCapacityUnits=5,WriteCapacityUnits=5

aws dynamodb create-table \
    --table-name orders \
    --attribute-definitions \
        AttributeName=order_id,AttributeType=S \
    --key-schema \
        AttributeName=order_id,KeyType=HASH \
    --provisioned-throughput \
        ReadCapacityUnits=5,WriteCapacityUnits=5
```

`users` テーブルには、ユーザーIDをハッシュキーとして使用します。 `orders` テーブルには、注文IDをハッシュキーとして使用します。プロビジョンドスループットは読み取り5、書き込み5に設定されています。必要に応じて、これらの値を調整してください。
create_dynamodb_tables()

## Step 4: Cognito 認証の設定

日本語訳:

Amazon Cognito を使用して、ユーザー認証を設定します。Cognito は AWS が提供する認証サービスで、ユーザープールとアイデンティティプールの 2 つの主要な機能があります。

1. **ユーザープール**: ユーザーディレクトリを作成し、ユーザーサインアップ、サインイン、アカウント確認などのユーザー管理機能を提供します。

2. **アイデンティティプール**: 認証済みユーザーと認証されていないユーザーの両方に対して、AWS サービスへのアクセス権を付与します。

この手順では、ユーザープールとアイデンティティプールの両方を設定します。

In [ ]:
def setup_cognito_auth():
    """Bedrock AgentCore Gateway の Cognito 認証をセットアップする"""
    cognito = boto3.client('cognito-idp', region_name='us-east-1')
    
    try:
        user_pool_response = cognito.create_user_pool(
            PoolName='bedrock-agentcore-gateway-dynamo-pool'
        )
        user_pool_id = user_pool_response['UserPool']['Id']
        
        import random
        import string
        domain_name = f"bedrock-agentcore-dynamo-{''.join(random.choices(string.ascii_lowercase + string.digits, k=8))}"
        
        try:
            cognito.create_user_pool_domain(
                Domain=domain_name,
                UserPoolId=user_pool_id
            )
        except:
            pass
        
        cognito.create_resource_server(
            UserPoolId=user_pool_id,
            Identifier='bedrock-agentcore-server',
            Name='BedrockCoreGatewayServer',
            Scopes=[
                {'ScopeName': 'read', 'ScopeDescription': 'Read access'},
                {'ScopeName': 'write', 'ScopeDescription': 'Write access'}
            ]
        )
        
        client_response = cognito.create_user_pool_client(
            UserPoolId=user_pool_id,
            ClientName='bedrock-agentcore-client',
            GenerateSecret=True,
            AllowedOAuthFlows=['client_credentials'],
            AllowedOAuthScopes=['bedrock-agentcore-server/read', 'bedrock-agentcore-server/write'],
            AllowedOAuthFlowsUserPoolClient=True,
            SupportedIdentityProviders=['COGNITO']
        )
        
        client_id = client_response['UserPoolClient']['ClientId']
        client_secret = client_response['UserPoolClient']['ClientSecret']
        
        auth_config = {
            'user_pool_id': user_pool_id,
            'client_id': client_id,
            'client_secret': client_secret,
            'discovery_url': f"https://cognito-idp.us-east-1.amazonaws.com/{user_pool_id}/.well-known/openid-configuration",
            'token_endpoint': f"https://{domain_name}.auth.us-east-1.amazoncognito.com/oauth2/token",
            'domain_name': domain_name
        }
        
        print(f"✅ Created Cognito configuration")
        return auth_config
        
    except Exception as e:
        print(f"❌ Error setting up Cognito: {e}")

auth_config = setup_cognito_auth()
print(f"Auth Configuration: {json.dumps(auth_config, indent=2)}")

## ステップ 5: DynamoDB を使用してバックエンド API Lambda 関数を作成する

日本語訳:

このステップでは、DynamoDB テーブルを作成し、Lambda 関数を作成して API Gateway にデプロイします。

1. DynamoDB テーブルを作成します。テーブル名は `notes` とし、パーティションキーは `noteId` (文字列型) とします。

2. `amplify/backend/function` ディレクトリに移動し、以下のコマンドを実行して新しい Lambda 関数を作成します。

```bash
amplify add function
```

次の質問に答えます。
- `Choose an option`: `Lambda function (serverless function)`
- `Provide an Function Name`: `AmplifyDataOperations`
- `Choose the function template that you want to use`: `CRUD function for Amazon DynamoDB`
- `Choose a DynamoDB data source option`: `Create a new DynamoDB table`
- `Provide table name`: `notes`

3. Lambda 関数のコードを更新して、`amplify/backend/function/AmplifyDataOperations/app.js` に以下のコードを追加します。

```js
const AWS = require('aws-sdk')
const docClient = new AWS.DynamoDB.DocumentClient()

const tableName = process.env.API_AMPLIFYAPP_NOTESTABLE_NAME

exports.handler = async (event) => {
    let body
    const httpMethod = event.httpMethod
    switch (httpMethod) {
        case 'GET':
            body = await getNote(event)
            break
        case 'POST':
            body = await createNote(event)
            break
        case 'PUT':
            body = await updateNote(event)
            break
        case 'DELETE':
            body = await deleteNote(event)
            break
        default:
            throw new Error(`Unsupported route: "${httpMethod}"`)
    }

    return {
        statusCode: 200,
        body: JSON.stringify(body)
    }
}

async function getNote(event) {
    const noteId = event.pathParameters.noteId

    const params = {
        TableName: tableName,
        Key: { noteId }
    }

    const data = await docClient.get(params).promise()
    if (!data.Item) {
        throw new Error(`Note with ID "${noteId}" not found.`)
    }

    return data.Item
}

async function createNote(event) {
    const body = JSON.parse(event.body)
    const noteId = body.noteId

    const params = {
        TableName: tableName,
        Item: body
    }

    await docClient.put(params).promise()

    return body
}

async function updateNote(event) {
    const body = JSON.parse(event.body)
    const noteId = event.pathParameters.noteId

    const params = {
        TableName: tableName,
        Key: { noteId },
        UpdateExpression: 'set #n = :note',
        ExpressionAttributeNames: { '#n': 'note' },
        ExpressionAttributeValues: { ':note': body.note }
    }

    await docClient.update(params).promise()

    return { noteId, ...body }
}

async function deleteNote(event) {
    const noteId = event.pathParameters.noteId

    const params = {
        TableName: tableName,
        Key: { noteId }
    }

    await docClient.delete(params).promise()

    return { noteId }
}
```

4. `amplify/backend/function/AmplifyDataOperations/package.json` ファイルを更新して、`aws-sdk` 依存関係を追加します。

```json
{
  "dependencies": {
    "aws-sdk": "^2.1313.0"
  }
}
```

5. 最後に、以下のコマンドを実行して API とリソースをデプロイします。

```bash
amplify push
```

これで、DynamoDB テーブルと CRUD 操作を行う Lambda 関数が作成されました。次のステップでは、フロントエンドアプリケーションを作成し、この API を使用します。

In [ ]:
# 強化された Lambda 関数コード (DynamoDB 統合付き)

日本語訳:

この Lambda 関数は、Amazon DynamoDB テーブルに新しいアイテムを追加し、既存のアイテムを更新および削除するための API を提供します。

`AWS_REGION` 環境変数は、使用する AWS リージョンを指定します。`TABLE_NAME` 環境変数は、アクセスする DynamoDB テーブルの名前を指定します。

`lambda_handler` 関数は、API Gateway からのイベントを受け取り、リクエストの `httpMethod` に基づいて適切なアクションを実行します。

- `GET` リクエストの場合、`get_item` 関数が呼び出され、テーブルから指定された `id` のアイテムを取得します。
- `PUT` リクエストの場合、`put_item` 関数が呼び出され、テーブルに新しいアイテムを追加するか、既存のアイテムを完全に置き換えます。
- `PATCH` リクエストの場合、`update_item` 関数が呼び出され、テーブル内の既存のアイテムを部分的に更新します。
- `DELETE` リクエストの場合、`delete_item` 関数が呼び出され、テーブルから指定された `id` のアイテムを削除します。

各関数は、適切な DynamoDB API を呼び出し、リクエストを処理します。レスポンスは JSON 形式で返されます。
lambda_code = '''
import json
import uuid
import boto3
from datetime import datetime
from decimal import Decimal

dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
orders_table = dynamodb.Table('FruitOrders')
inventory_table = dynamodb.Table('FruitInventory')

def determine_tool_from_arguments(event):
    if not event:
        return "list_fruits_tool"
    if "customer_name" in event or "items" in event:
        return "create_order_tool"
    elif "order_id" in event:
        return "get_order_tool"
    return "list_fruits_tool"

def list_fruits_tool(arguments):
    try:
        response = inventory_table.scan()
        fruits = []
        for item in response['Items']:
            fruits.append({
                "name": item['fruit_name'],
                "price": float(item['price']),
                "unit": item['unit'],
                "stock": int(item['stock'])
            })
        return {"fruits": fruits, "status": "success"}
    except Exception as e:
        return {"error": str(e), "status": "error"}

def create_order_tool(arguments):
    try:
        items = arguments.get('items', [])
        customer_name = arguments.get('customer_name', 'Anonymous')
        
        if not items:
            return {"error": "No items provided", "status": "error"}
        
        order_id = str(uuid.uuid4())[:8]
        total_cost = Decimal('0')
        order_items = []
        
        for item in items:
            fruit_name = item.get('name', '').lower()
            quantity = item.get('quantity', 1)
            
            try:
                fruit_response = inventory_table.get_item(
                    Key={'fruit_name': fruit_name}
                )
                if 'Item' not in fruit_response:
                    return {"error": f"Fruit '{fruit_name}' not found", "status": "error"}
                
                fruit = fruit_response['Item']
                if int(fruit['stock']) < quantity:
                    return {"error": f"Insufficient stock for {fruit_name}. Available: {fruit['stock']}", "status": "error"}
                
                unit_price = Decimal(str(fruit['price']))
                item_cost = unit_price * Decimal(str(quantity))
                total_cost += item_cost
                
                order_items.append({
                    "name": fruit['fruit_name'],
                    "quantity": quantity,
                    "unit_price": unit_price,
                    "unit": fruit['unit'],
                    "total_price": item_cost
                })
                
                inventory_table.update_item(
                    Key={'fruit_name': fruit_name},
                    UpdateExpression='SET stock = stock - :qty',
                    ExpressionAttributeValues={':qty': quantity}
                )
                
            except Exception as e:
                return {"error": f"Error processing {fruit_name}: {str(e)}", "status": "error"}
        
        orders_table.put_item(Item={
            'order_id': order_id,
            'customer_name': customer_name,
            'items': order_items,
            'total_cost': total_cost,
            'status': 'pending',
            'created_at': datetime.now().isoformat()
        })
        
        order = {
            "order_id": order_id,
            "customer_name": customer_name,
            "items": [
                {
                    "name": item["name"],
                    "quantity": item["quantity"],
                    "unit_price": float(item["unit_price"]),
                    "unit": item["unit"],
                    "total_price": float(item["total_price"])
                }
                for item in order_items
            ],
            "total_cost": float(total_cost),
            "status": "pending",
            "created_at": datetime.now().isoformat()
        }
        
        return {"order": order, "status": "success"}
        
    except Exception as e:
        return {"error": str(e), "status": "error"}

def get_order_tool(arguments):
    try:
        order_id = arguments.get('order_id')
        if not order_id:
            return {"error": "Order ID is required", "status": "error"}
        
        response = orders_table.get_item(Key={'order_id': order_id})
        if 'Item' not in response:
            return {"error": f"Order {order_id} not found", "status": "error"}
        
        order = response['Item']
        order['total_cost'] = float(order['total_cost'])
        for item in order.get('items', []):
            if 'unit_price' in item:
                item['unit_price'] = float(item['unit_price'])
            if 'total_price' in item:
                item['total_price'] = float(item['total_price'])
        
        return {"order": order, "status": "success"}
        
    except Exception as e:
        return {"error": str(e), "status": "error"}

def lambda_handler(event, context):
    try:
        tool_name = determine_tool_from_arguments(event)
        
        tools = {
            'list_fruits_tool': list_fruits_tool,
            'create_order_tool': create_order_tool,
            'get_order_tool': get_order_tool
        }
        
        tool_function = tools.get(tool_name)
        if not tool_function:
            return {"error": f"Unknown tool: {tool_name}", "status": "error"}
        
        return tool_function(event)
        
    except Exception as e:
        return {"error": str(e), "status": "error"}
'''

def create_enhanced_lambda():
    lambda_client = boto3.client('lambda', region_name='us-east-1')
    
    with open('enhanced_lambda_function.py', 'w') as f:
        f.write(lambda_code)
    
    with zipfile.ZipFile('enhanced_fruit_orders_lambda.zip', 'w') as zip_file:
        zip_file.write('enhanced_lambda_function.py', 'lambda_function.py')
    
    try:
        with open('enhanced_fruit_orders_lambda.zip', 'rb') as zip_file:
            response = lambda_client.create_function(
                FunctionName='enhanced-fruit-order-dynamo',
                Runtime='python3.13',
                Role=gateway_role_arn,
                Handler='lambda_function.lambda_handler',
                Code={'ZipFile': zip_file.read()},
                Description='Enhanced fruit ordering system with DynamoDB',
                Timeout=30
            )
        
        print(f"✅ Created Enhanced Lambda: {response['FunctionArn']}")
        return response['FunctionArn']
        
    except Exception as e:
        if 'already exists' in str(e):
            account_id = boto3.client('sts').get_caller_identity()['Account']
            lambda_arn = f"arn:aws:lambda:us-east-1:{account_id}:function:enhanced-fruit-order-dynamo"
            print(f"✅ Lambda function already exists: {lambda_arn}")
            return lambda_arn
        else:
            print(f"❌ Error: {e}")
            return None
    finally:
        if os.path.exists('enhanced_lambda_function.py'):
            os.remove('enhanced_lambda_function.py')
        if os.path.exists('enhanced_fruit_orders_lambda.zip'):
            os.remove('enhanced_fruit_orders_lambda.zip')

enhanced_lambda_arn = create_enhanced_lambda()
print(f"Enhanced Lambda ARN: {enhanced_lambda_arn}")

## ステップ 6: Bedrock AgentCore Gateway を作成する

日本語訳:

Bedrock AgentCore Gateway は、エージェントとクラウドサービスの間の通信を可能にする重要なコンポーネントです。このゲートウェイを作成するには、次の手順に従ってください。

1. Bedrock CLI を使用して新しい Bedrock プロジェクトを作成します: `bedrock new gateway my-gateway`
2. `my-gateway` ディレクトリに移動します: `cd my-gateway`
3. `gateway.yaml` ファイルを編集して、ゲートウェイの設定を行います。
4. ゲートウェイを起動します: `bedrock start`

ゲートウェイが正常に起動したら、エージェントとクラウドサービスの間の通信が可能になります。エージェントは `AgentCore.connect()` 関数を使用してゲートウェイに接続し、クラウドサービスと対話できます。

ゲートウェイの設定オプションの詳細については、Bedrock のドキュメントを参照してください。

In [ ]:
def create_bedrock_agentcore_gateway(role_arn, auth_config):
    以下が日本語訳になります。

"""Bedrock AgentCore Gateway を作成する"""

技術的な用語は翻訳せずにそのまま残しました。半角英数字の前後には半角スペースを挿入しています。
    client = boto3.client(
        'bedrock-agentcore-control',
        region_name='us-east-1',
        endpoint_url="https://bedrock-agentcore-control.us-east-1.amazonaws.com"
    )
    
    gateway_auth_config = {
        "customJWTAuthorizer": {
            "allowedClients": [auth_config['client_id']],
            "discoveryUrl": auth_config['discovery_url']
        }
    }
    
    try:
        response = client.create_gateway(
            name=f'fruit-orders-dynamo-gateway-{int(time.time())%1000000}',
            roleArn=role_arn,
            protocolType='MCP',
            authorizerType='CUSTOM_JWT',
            authorizerConfiguration=gateway_auth_config,
            description='Enhanced Bedrock Core Agent Gateway with Lambda integration'
        )
        
        gateway_id = response.get('gatewayId')
        print(f"✅ Created Gateway: {gateway_id}")
        return gateway_id
        
    except Exception as e:
        print(f"❌ Error creating gateway: {e}")
        return None

if gateway_role_arn and auth_config:
    gateway_id = create_bedrock_agentcore_gateway(gateway_role_arn, auth_config)
    print(f"Gateway ID: {gateway_id}")
else:
    print("⚠️ Gateway role ARN and auth config required")
    gateway_id = None

## ステップ 7: 拡張スキーマを使用してゲートウェイ Lambda ターゲットを作成する

日本語訳:

このステップでは、拡張スキーマを使用して API Gateway の Lambda 統合を設定します。拡張スキーマを使用すると、リクエストペイロードとレスポンスペイロードの構造を定義できます。これにより、Lambda 関数で受け取るデータの形式を制御し、返されるデータの形式を指定できます。

1. AWS Lambda コンソールで、作成した `lambda-proxy` 関数を選択します。
2. 「トリガーの追加」をクリックします。
3. 「トリガーの選択」ドロップダウンから `API Gateway` を選択します。
4. 「API の作成」をクリックします。
5. 「API の種類」で `HTTP API` を選択します。
6. 「セキュリティ」で `オープン` を選択します。
7. 「API の新規作成」をクリックします。
8. 「統合の設定」セクションで、「Lambda 関数」として作成した `lambda-proxy` 関数を選択します。
9. 「拡張スキーマの使用」チェックボックスをオンにします。
10. リクエストペイロードのモデルを定義します (例: `{ "name": "$input.json('$.name')" }`)。
11. レスポンスペイロードのモデルを定義します (例: `{ "result": "$input.json('$.result')" }`)。
12. 「API の作成」をクリックします。

これで、拡張スキーマを使用して API Gateway の Lambda 統合が設定されました。リクエストとレスポンスのペイロードの構造が定義されているため、Lambda 関数で受け取るデータと返すデータの形式を制御できます。

In [ ]:
def create_enhanced_gateway_target(gateway_id, lambda_arn):
    client = boto3.client(
        'bedrock-agentcore-control',
        region_name='us-east-1',
        endpoint_url="https://bedrock-agentcore-control.us-east-1.amazonaws.com"
    )
    
    lambda_target_config = {
        "mcp": {
            "lambda": {
                "lambdaArn": lambda_arn,
                "toolSchema": {
                    "inlinePayload": [
                        {
                            "name": "list_fruits_tool",
                            "description": "Lists all available fruits with prices and stock levels",
                            "inputSchema": {"type": "object", "properties": {}, "required": []}
                        },
                        {
                            "name": "create_order_tool",
                            "description": "Creates a new fruit order and updates inventory",
                            "inputSchema": {
                                "type": "object",
                                "properties": {
                                    "customer_name": {
                                        "type": "string", 
                                    },
                                    "items": {
                                        "type": "array",
                                        "items": {
                                            "type": "object",
                                            "properties": {
                                                "name": {
                                                    "type": "string", 
                                                },
                                                "quantity": {
                                                    "type": "number", 
                                                }
                                            },
                                            "required": ["name", "quantity"]
                                        }
                                    }
                                },
                                "required": ["customer_name", "items"]
                            }
                        },
                        {
                            "name": "get_order_tool",
                            "description": "Gets order details by ID from DynamoDB",
                            "inputSchema": {
                                "type": "object",
                                "properties": {
                                    "order_id": {
                                        "type": "string",
                                    }
                                },
                                "required": ["order_id"]
                            }
                        }
                    ]
                }
            }
        }
    }
    
    try:
        response = client.create_gateway_target(
            gatewayIdentifier=gateway_id,
            name='EnhancedFruitOrdersTarget',
            description='Enhanced Lambda target with DynamoDB integration',
            targetConfiguration=lambda_target_config,
            credentialProviderConfigurations=[{"credentialProviderType": "GATEWAY_IAM_ROLE"}]
        )
        
        target_id = response['targetId']
        print(f"✅ Created Target: {target_id}")
        return target_id
        
    except Exception as e:
        print(f"❌ Error creating target: {e}")
        return None

if gateway_id and enhanced_lambda_arn:
    target_id = create_enhanced_gateway_target(gateway_id, enhanced_lambda_arn)
    print(f"Target ID: {target_id}")
else:
    print("⚠️ Gateway ID and Lambda ARN required")
    target_id = None

## ステップ 8: List と Ivoke を使ってゲートウェイをテストする

日本語訳:

ゲートウェイが正しく設定されたことを確認するには、 `gcloud` コマンドラインツールを使って API を一覧表示し、呼び出すことができます。

1. 次のコマンドを実行して、プロジェクトで有効になっている API を一覧表示します。

```
gcloud services list --enabled
```

出力には、有効になっている API の一覧が表示されます。 `servicenetworking.googleapis.com` が一覧にあることを確認してください。

2. 次のコマンドを実行して、 `servicenetworking.googleapis.com` API を呼び出します。

```
gcloud services vpc-peerings list --network=default
```

このコマンドは、デフォルトの VPC ネットワークのピアリングを一覧表示します。出力は空の場合がありますが、エラーは表示されません。

ゲートウェイが正しく設定されていれば、これらのコマンドは問題なく実行できます。問題がある場合は、前の手順を確認してください。

In [ ]:
def generate_bearer_token(auth_config):
    data = urllib.parse.urlencode({
        "grant_type": "client_credentials",
        "client_id": auth_config['client_id'],
        "client_secret": auth_config['client_secret']
    }).encode('utf-8')
    
    try:
        req = urllib.request.Request(
            auth_config['token_endpoint'],
            data=data,
            headers={"Content-Type": "application/x-www-form-urlencoded"}
        )
        
        with urllib.request.urlopen(req, timeout=10) as response:
            if response.status == 200:
                result = json.loads(response.read().decode('utf-8'))
                return result.get('access_token')
    except Exception as e:
        print(f"Error generating token: {e}")
    return None

def test_enhanced_gateway(gateway_id, target_id, auth_config):
    bearer_token = generate_bearer_token(auth_config)
    if not bearer_token:
        print("❌ Failed to generate token")
        return
    
    mcp_endpoint = f"https://{gateway_id}.gateway.bedrock-agentcore.us-east-1.amazonaws.com/mcp"
    
    # Test 1: 在庫レベルがある果物のリストを表示

日本語訳:

# Test 1: 在庫レベルのある果物のリストを表示

fruits = ['apple', 'banana', 'orange', 'kiwi']
stock = [5, 0, 12, 3]

for i in range(len(fruits)):
    fruit = fruits[i]
    level = stock[i]
    print(f'{ fruit }: { level }')
    print("\n🧪 Test 1: List fruits with stock levels")
    mcp_request = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "tools/call",
        "params": {
            "name": "EnhancedFruitOrdersTarget___list_fruits_tool",
            "arguments": {}
        }
    }
    
    try:
        req = urllib.request.Request(
            mcp_endpoint,
            data=json.dumps(mcp_request).encode('utf-8'),
            headers={
                'Content-Type': 'application/json',
                'Authorization': f'Bearer {bearer_token}'
            }
        )
        
        with urllib.request.urlopen(req, timeout=30) as response:
            if response.status == 200:
                result = json.loads(response.read().decode('utf-8'))
                print("✅ List fruits successful!")
                print(json.dumps(result, indent=2))
    except Exception as e:
        print(f"❌ Error: {e}")
    
    # Test 2: 在庫更新を伴う注文の作成

日本語訳:

# Test 2: 在庫更新を伴う注文の作成
    print("\n🧪 Test 2: Create order with inventory update")
    create_order_request = {
        "jsonrpc": "2.0",
        "id": 2,
        "method": "tools/call",
        "params": {
            "name": "EnhancedFruitOrdersTarget___create_order_tool",
            "arguments": {
                "customer_name": "Alice Johnson",
                "items": [
                    {"name": "apple", "quantity": 3},
                    {"name": "banana", "quantity": 2}
                ]
            }
        }
    }
    
    try:
        req = urllib.request.Request(
            mcp_endpoint,
            data=json.dumps(create_order_request).encode('utf-8'),
            headers={
                'Content-Type': 'application/json',
                'Authorization': f'Bearer {bearer_token}'
            }
        )
        
        with urllib.request.urlopen(req, timeout=30) as response:
            if response.status == 200:
                result = json.loads(response.read().decode('utf-8'))
                print("✅ Create order successful!")
                print(json.dumps(result, indent=2))
                
                # 次のテストのための order ID を抽出する

日本語訳:

# 次のテストのための order ID を抽出します

order_id = db.orders.find_one({ 'status': 'open' })['_id']

print(f'Extracted order ID: { order_id }')
                if 'result' in result and 'content' in result['result']:
                    content = result['result']['content'][0]['text']
                    order_data = json.loads(content)
                    if 'order' in order_data:
                        order_id = order_data['order']['order_id']
                        print(f"\n📝 Order ID for testing: {order_id}")
                        return order_id
    except Exception as e:
        print(f"❌ Error: {e}")
    
    return None

# Test the enhanced gateway

# 強化されたゲートウェイをテストする

日本語訳:

# 強化された gateway をテストします
if gateway_id and target_id and auth_config:
    test_order_id = test_enhanced_gateway(gateway_id, target_id, auth_config)
else:
    print("⚠️ Gateway ID, Target ID, and auth config required for testing")
    test_order_id = None

以下が日本語訳になります。

## ステップ 9: MCP クライアントとして機能する Bedrock エージェントを作成する

半角英数字の前後にスペースを挿入し、コード、コマンド、変数名、関数名などの技術的な用語はそのまま残しました。

In [ ]:
def create_enhanced_bedrock_agent():
    bedrock = boto3.client('bedrock-agent', region_name='us-east-1')
    
    try:
        agent_response = bedrock.create_agent(
            agentName='enhanced-fruit-ordering-dynamo-agent',
            description='Enhanced AI agent for fruit ordering with DynamoDB persistence',
            foundationModel='anthropic.claude-3-sonnet-20240229-v1:0',
            instruction='''You are an advanced fruit ordering assistant with access to a persistent inventory system. You can:

1. **List Available Fruits**: Show all fruits with current prices and stock levels
2. **Create Orders**: Process customer orders and automatically update inventory
3. **Track Orders**: Retrieve order details using order IDs
4. **Inventory Management**: Check stock levels and prevent overselling

Key Features:
- Real-time inventory tracking with DynamoDB
- Automatic stock deduction when orders are placed
- Persistent order storage with customer information
- Stock validation to prevent overselling

Always be helpful, provide clear information about availability, and confirm order details including total cost.''',
            agentResourceRoleArn=gateway_role_arn,
            idleSessionTTLInSeconds=1800
        )
        
        agent_id = agent_response['agent']['agentId']
        print(f"✅ Created Enhanced Agent: {agent_id}")
        return agent_id
        
    except Exception as e:
        print(f"❌ Error creating agent: {e}")
        return None

enhanced_agent_id = create_enhanced_bedrock_agent()
print(f"Enhanced Agent ID: {enhanced_agent_id}")

## ステップ 10: Bedrock Agent 用の Bridge Lambda を作成する

Bedrock AgentCore Gateway への Bedrock Agent の呼び出しをブリッジする Lambda 関数を作成します。

1. AWS Lambda コンソールを開き、 `Create function` をクリックします。
2. `Author from scratch` を選択します。
3. 関数名に `BedrockAgentBridge` と入力します。
4. ランタイムに `Python 3.9` を選択します。
5. `Change default execution role` をクリックし、 `Create a new role with basic Lambda permissions` を選択します。
6. `Function code` セクションで、 `lambda_function.py` にサンプルコードを貼り付けます。
7. `Configuration` タブで、 `General configuration` の `Timeout` を `30` 秒に設定します。
8. `Environment variables` セクションで、以下の環境変数を追加します:
   - `AGENT_CORE_GATEWAY_URL` - Bedrock AgentCore Gateway の URL (例: `https://xyz.execute-api.us-west-2.amazonaws.com/prod`)
9. `Save` をクリックして Lambda 関数を作成します。

これで、Bedrock Agent からの呼び出しを Bedrock AgentCore Gateway にブリッジする Lambda 関数が作成されました。

In [ ]:
def create_bridge_lambda(gateway_id, target_id, auth_config):
    bridge_code = f'''
import json
import urllib.request
import urllib.parse

def generate_bearer_token():
    data = urllib.parse.urlencode({{
        "grant_type": "client_credentials",
        "client_id": "{auth_config['client_id']}",
        "client_secret": "{auth_config['client_secret']}"
    }}).encode('utf-8')
    
    try:
        req = urllib.request.Request(
            "{auth_config['token_endpoint']}",
            data=data,
            headers={{"Content-Type": "application/x-www-form-urlencoded"}}
        )
        
        with urllib.request.urlopen(req, timeout=10) as response:
            if response.status == 200:
                result = json.loads(response.read().decode('utf-8'))
                return result.get('access_token')
    except Exception as e:
        print(f"Error generating token: {{e}}")
    return None

def lambda_handler(event, context):
    gateway_endpoint = "https://{gateway_id}.gateway.bedrock-agentcore.us-east-1.amazonaws.com/mcp"
    
    bearer_token = generate_bearer_token()
    if not bearer_token:
        return {{
            "messageVersion": "1.0",
            "response": {{
                "actionGroup": event.get('actionGroup', ''),
                "function": event.get('function', ''),
                "functionResponse": {{
                    "responseBody": {{"TEXT": {{"body": "Failed to generate token"}}}}
                }}
            }}
        }}
    
    function_name = event.get('function')
    parameters = event.get('parameters', [])
    
    param_dict = {{}}
    for param in parameters:
        param_dict[param['name']] = param['value']
    
    tool_mapping = {{
        'list_fruits': 'EnhancedFruitOrdersTarget___list_fruits_tool',
        'create_order': 'EnhancedFruitOrdersTarget___create_order_tool',
        'get_order': 'EnhancedFruitOrdersTarget___get_order_tool'
    }}
    
    gateway_tool = tool_mapping.get(function_name)
    if not gateway_tool:
        return {{
            "messageVersion": "1.0",
            "response": {{
                "actionGroup": event.get('actionGroup', ''),
                "function": function_name,
                "functionResponse": {{
                    "responseBody": {{"TEXT": {{"body": f"Unknown function: {{function_name}}"}}}}
                }}
            }}
        }}
    
    arguments = {{}}
    if function_name == 'create_order':
        arguments = {{
            'customer_name': param_dict.get('customer_name', 'Customer'),
            'items': json.loads(param_dict.get('items', '[]'))
        }}
    elif function_name == 'get_order':
        arguments = {{'order_id': param_dict.get('order_id')}}
    
    mcp_request = {{
        "jsonrpc": "2.0",
        "id": 1,
        "method": "tools/call",
        "params": {{"name": gateway_tool, "arguments": arguments}}
    }}
    
    try:
        req = urllib.request.Request(
            gateway_endpoint,
            data=json.dumps(mcp_request).encode('utf-8'),
            headers={{
                'Content-Type': 'application/json',
                'Authorization': f'Bearer {{bearer_token}}'
            }}
        )
        
        with urllib.request.urlopen(req, timeout=30) as response:
            if response.status == 200:
                result = json.loads(response.read().decode('utf-8'))
                if 'result' in result and 'content' in result['result']:
                    content_text = result['result']['content'][0].get('text', '')
                    try:
                        parsed_result = json.loads(content_text)['response']['payload']
                        response_text = json.dumps(parsed_result, indent=2)
                    except:
                        response_text = content_text
                else:
                    response_text = json.dumps(result, indent=2)
            else:
                response_text = f"Gateway error: {{response.status}}"
    except Exception as e:
        response_text = f"Error: {{str(e)}}"
    
    return {{
        "messageVersion": "1.0",
        "response": {{
            "actionGroup": event.get('actionGroup', ''),
            "function": function_name,
            "functionResponse": {{
                "responseBody": {{"TEXT": {{"body": response_text}}}}
            }}
        }}
    }}
'''
    
    lambda_client = boto3.client('lambda', region_name='us-east-1')
    
    with open('bridge_function.py', 'w') as f:
        f.write(bridge_code)
    
    with zipfile.ZipFile('bridge_lambda.zip', 'w') as zip_file:
        zip_file.write('bridge_function.py', 'lambda_function.py')
    
    try:
        with open('bridge_lambda.zip', 'rb') as zip_file:
            response = lambda_client.create_function(
                FunctionName='enhanced-bedrock-gateway-bridge',
                Runtime='python3.13',
                Role=gateway_role_arn,
                Handler='lambda_function.lambda_handler',
                Code={'ZipFile': zip_file.read()},
                Description='Bridge between Bedrock Agent and Bedrock AgentCore Gateway',
                Timeout=30
            )
        
        bridge_arn = response['FunctionArn']
        print(f"✅ Created Bridge Lambda: {bridge_arn}")
        
        # Add permission for Bedrock

日本語訳:

# Bedrock の許可を追加

Bedrock は、 Linux カーネルの セキュリティモデルの 1 つです。 Bedrock を使用するには、以下のコマンドを実行して、適切な許可を付与する必要があります。

```
sysctl -w kernel.grsecurity.prio.unmute=1
```

このコマンドを実行すると、 Bedrock が有効になり、 `CONFIG_GRKERNSEC_PRIO_UNMUTE` オプションが有効になります。このオプションにより、 Bedrock は、プロセスの実行時に、そのプロセスのセキュリティポリシーを変更できるようになります。

Bedrock は、プロセスの実行時に、そのプロセスに対して、より制限の厳しいセキュリティポリシーを適用することができます。これにより、悪意のあるコードの実行を防ぐことができます。
        lambda_client.add_permission(
            FunctionName='enhanced-bedrock-gateway-bridge',
            StatementId='bedrock-agent-invoke',
            Action='lambda:InvokeFunction',
            Principal='bedrock.amazonaws.com'
        )
        
        return bridge_arn
        
    except Exception as e:
        if 'already exists' in str(e):
            account_id = boto3.client('sts').get_caller_identity()['Account']
            bridge_arn = f"arn:aws:lambda:us-east-1:{account_id}:function:enhanced-bedrock-gateway-bridge"
            print(f"✅ Bridge Lambda already exists: {bridge_arn}")
            return bridge_arn
        else:
            print(f"❌ Error: {e}")
            return None
    finally:
        if os.path.exists('bridge_function.py'):
            os.remove('bridge_function.py')
        if os.path.exists('bridge_lambda.zip'):
            os.remove('bridge_lambda.zip')

# Create Bridge Lambda

この Lambda 関数は、API Gateway から呼び出されて、リクエストを Amazon EventBridge に転送します。EventBridge は、AWS サービスから送信されたイベントを受信し、ルーティングするサーバーレスイベントバスです。

```python
import json
import boto3

eventbridge = boto3.client('events')

def lambda_handler(event, context):
    print('Received event: ' + json.dumps(event, indent=2))

    put_response = eventbridge.put_events(
        Entries=[
            {
                'Detail': json.dumps(event),
                'DetailType': 'ApiGatewayProxy',
                'Resources': [],
                'Source': 'api.gateway'
            }
        ]
    )

    print('PutEvents Response: ' + json.dumps(put_response, indent=2))

    return {
        'statusCode': 200,
        'body': json.dumps('Success!')
    }
```

この Lambda 関数は、以下の手順を実行します:

1. API Gateway からのイベントを受信し、ログに出力します。
2. `put_events` を使って、受信したイベントを EventBridge に転送します。
3. EventBridge からのレスポンスをログに出力します。
4. HTTP 200 レスポンスと 'Success!' メッセージを返します。

この Lambda 関数は、API Gateway を EventBridge に接続するブリッジの役割を果たします。API Gateway からのリクエストを EventBridge に渡すことで、他の AWS サービスがそのイベントを受信し、処理することができます。
if gateway_id and target_id and auth_config:
    bridge_arn = create_bridge_lambda(gateway_id, target_id, auth_config)
    print(f"Bridge Lambda ARN: {bridge_arn}")
else:
    print("⚠️ Gateway ID, Target ID, and auth config required")
    bridge_arn = None

## ステップ 11: エージェントの作成を待機して確認する

エージェントが作成されるのを待ち、その状態を確認します。

```python
import time

while True:
    response = client.get_agent(
        agent_id=agent_id,
        headers=headers
    )
    
    if response["status"] == "Running":
        print("Agent created successfully!")
        break
    
    print(f"Agent status: {response['status']}")
    time.sleep(10)
```

上記の Python コードでは、 `client.get_agent()` を使用してエージェントの状態を定期的に確認しています。エージェントの状態が `"Running"` になったら、作成が成功したことを示すメッセージを出力し、ループを抜けます。そうでない場合は、現在の状態を出力し、10 秒間スリープした後にループを続行します。

In [ ]:
def wait_for_agent_creation(agent_id, max_wait=300):
    翻訳するテキスト:
"""Wait for agent to be created and ready"""

日本語訳:
""" agent が作成され、準備ができるのを待ちます """
    if not agent_id:
        print("⚠️ No agent ID provided")
        return False
        
    bedrock = boto3.client('bedrock-agent', region_name='us-east-1')
    
    print(f"⏳ Waiting for agent {agent_id} to be ready...")
    
    start_time = time.time()
    while time.time() - start_time < max_wait:
        try:
            response = bedrock.get_agent(agentId=agent_id)
            status = response['agent']['agentStatus']
            
            print(f"Agent status: {status}")
            
            if status == 'NOT_PREPARED':
                print("✅ Agent created successfully and ready for action groups")
                return True
            elif status == 'FAILED':
                print("❌ Agent creation failed")
                return False
            
            time.sleep(10)
            
        except Exception as e:
            print(f"Error checking agent status: {e}")
            time.sleep(10)
    
    print("⚠️ Timeout waiting for agent creation")
    return False

# Wait for agent creation

agent の作成を待ちます。
agent_ready = wait_for_agent_creation(enhanced_agent_id)
print(f"Agent ready: {agent_ready}")

## ステップ 12: アクション グループの関連付け

アクション グループを作成し、Bedrock Agent に関連付けます。

In [ ]:
def create_action_group(agent_id, bridge_lambda_arn):
    if not agent_id or not bridge_lambda_arn:
        print("⚠️ Agent ID and Bridge Lambda ARN required")
        return None
        
    bedrock = boto3.client('bedrock-agent', region_name='us-east-1')
    
    try:
        action_group_response = bedrock.create_agent_action_group(
            agentId=agent_id,
            agentVersion='DRAFT',
            actionGroupName='EnhancedFruitOrdersActions',
            description='Enhanced actions for fruit ordering with DynamoDB',
            actionGroupExecutor={'lambda': bridge_lambda_arn},
            functionSchema={
                'functions': [
                    {
                        'name': 'list_fruits',
                        'description': 'List available fruits with prices and stock levels'
                    },
                    {
                        'name': 'create_order',
                        'description': 'Create a new fruit order with inventory update',
                        'parameters': {
                            'customer_name': {'type': 'string', 'description': 'Customer name'},
                            'items': {'type': 'string', 'description': 'JSON array of items with name and quantity'}
                        }
                    },
                    {
                        'name': 'get_order',
                        'description': 'Get order details by ID from DynamoDB',
                        'parameters': {
                            'order_id': {'type': 'string', 'description': 'Order ID'}
                        }
                    }
                ]
            }
        )
        
        action_group_id = action_group_response['agentActionGroup']['actionGroupId']
        print(f"✅ Created Action Group: {action_group_id}")
        
        # Prepare agent

エージェントの準備

agent = Agent( )
agent.load_data(data_file)
agent.add_behavior(Behavior1, parameters)
agent.add_behavior(Behavior2, parameters)

print("Agent is ready.")
        bedrock.prepare_agent(agentId=agent_id)
        print(f"✅ Agent prepared with action group")
        
        return action_group_id
        
    except Exception as e:
        print(f"❌ Error creating action group: {e}")
        return None

# アクショングループを作成する

日本語訳:

この手順では、 Action Group を作成します。 Action Group は、特定のアラートルールがトリガーされたときに呼び出されるカスタマイズ可能な通知を定義するものです。通知の種類には、電子メール、 SMS、音声通話、Azure アプリの論理アプリ、Webhook、ITSM ツール (ServiceNow など) があります。

1. Azure ポータルで、 [ モニター ] をクリックします。

2. [ アラート ] セクションで、[ アクショングループの管理 ] をクリックします。

3. [ アクショングループの追加 ] をクリックし、アクショングループの詳細を入力します。

    a. 一意の名前を入力します。例: [ contoso-opsgroupr ]。

    b. 短い名前を入力します。例: [ opsgroupr ]。

    c. Azure サブスクリプションを選択します。

    d. リソースグループを選択するか、新しいリソースグループを作成します。

4. [ 次へ: 通知 ] をクリックします。

5. [ 通知の種類 ] ドロップダウンリストから、通知の種類を選択します。オプションは、電子メール、 SMS、音声通話、Azure アプリの論理アプリ、Webhook、ITSM ツールです。

6. 通知の詳細を入力します。

7. [ OK ] をクリックして通知を追加します。

8. 必要に応じて、さらに通知を追加します。

9. [ 確認と作成 ] をクリックします。

10. 入力内容を確認し、[ 作成 ] をクリックしてアクショングループを作成します。

これで、アクショングループが作成されました。次に、このアクショングループをアラートルールに関連付ける必要があります。
if agent_ready and bridge_arn:
    action_group_id = create_action_group(enhanced_agent_id, bridge_arn)
    print(f"Action Group ID: {action_group_id}")
else:
    print("⚠️ Agent must be ready and Bridge Lambda ARN required")
    action_group_id = None

## ステップ 13: Lambda をターゲットとした Bedrock AgentCore Gateway との Bedrock エージェントのエンドツーエンドフローのテスト

Bedrock エージェントを呼び出すことで、完全な統合をテストします。

In [ ]:
def test_enhanced_bedrock_agent(agent_id):
    bedrock_runtime = boto3.client('bedrock-agent-runtime', region_name='us-east-1')
    
    # エージェントの状態を最初に確認する

日本語訳:

エージェントの状態を最初に確認します。

$ agent status

エージェントが実行中の場合は、次のように表示されます。

pid 12345, uptime 2 days, status running, listening on 0.0.0.0:8000

エージェントが実行されていない場合は、次のように表示されます。

Agent not running

エージェントを起動するには、次のコマンドを実行します。

$ agent start

エージェントが正常に起動すると、次のように表示されます。

Starting agent on 0.0.0.0:8000
Agent started with pid 12345
    bedrock = boto3.client('bedrock-agent', region_name='us-east-1')
    try:
        agent_info = bedrock.get_agent(agentId=agent_id)
        agent_status = agent_info['agent']['agentStatus']
        print(f"🔍 Agent status: {agent_status}")
        
        if agent_status != 'PREPARED':
            print(f"⚠️ Agent not prepared. Current status: {agent_status}")
            return
    except Exception as e:
        print(f"❌ Error checking agent: {e}")
        return
    
    test_queries = [
        "What fruits are available?",
        "Create an order for Bob with fruit as apple and quantity as 2"
    ]
    
    for query in test_queries:
        print(f"\n🤔 User: {query}")
        
        try:
            response = bedrock_runtime.invoke_agent(
                agentId=agent_id,
                agentAliasId='TSTALIASID',
                sessionId=f'test-session-{int(time.time())}',
                inputText=query
            )
            
            full_response = ""
            for event in response['completion']:
                if 'chunk' in event:
                    chunk = event['chunk']
                    if 'bytes' in chunk:
                        full_response += chunk['bytes'].decode('utf-8')
            
            print(f"🤖 Enhanced Agent: {full_response}")
            
        except Exception as e:
            print(f"❌ Error: {e}")

# テストの強化されたベースロックエージェント

あなたは 翻訳 の 専門家 です 。 以下 の テキスト を 英語 から 日本語 に 翻訳 して ください 。
半角 英数字 の 前後 には 半角 スペース を 挿入 する 。 コード や コマンド 、 変数名 、 関数名 など の 技術的 な 用語 は 翻訳 せず 、 そのまま 残して ください 。

翻訳 する テキスト :
# Test the Enhanced Bedrock Agent

To test the enhanced agent, you will need to have the Bedrock Preview channel installed. Follow the instructions here to install the preview: https://aka.ms/openai-minecraft-preview

Once you have the preview installed, create a new world and enable the "Enhanced Bedrock Agent" game mode from the Game menu.

You can then use the `/bragent` command to interact with the agent. For example:

`/bragent help` - Get help on available commands
`/bragent build house` - Build a simple house
`/bragent goto x y z` - Teleport the agent to specific coordinates

The enhanced agent has more capabilities than the original Bedrock agent, including the ability to build more complex structures, gather and craft items, and navigate the world more intelligently.

Have fun exploring the new agent capabilities! Let us know if you have any feedback.
if enhanced_agent_id:
    print("Testing Enhanced Bedrock Agent with DynamoDB integration...")
    test_enhanced_bedrock_agent(enhanced_agent_id)
else:
    print("⚠️ Agent ID required for testing")

## 🎉 概要

### アーキテクチャ:
```
User Query → Bedrock Agent → Bridge Lambda → Bedrock AgentCore Gateway → Enhanced Lambda → DynamoDB
```

### データベーススキーマ:
- **FruitOrders**: order_id, customer_name, items, total_cost, status, created_at
- **FruitInventory**: fruit_name, price, unit, stock

## 🧹 リソースのクリーンアップ

依存関係の問題を避けるため、以下の各セクションを順番に実行して、特定のリソースをクリーンアップします。

### Step 1: Delete Bedrock Agent

日本語訳:

### ステップ 1: Bedrock Agent を削除する

まず、既存の Bedrock Agent を削除する必要があります。これを行うには、次のコマンドを実行してください。

```
bedrock uninstall
```

このコマンドは、Bedrock Agent とそのすべての関連ファイルを完全に削除します。削除が完了したら、次のステップに進みます。

In [ ]:
bedrock = boto3.client('bedrock-agent', region_name='us-east-1')

try:
    if 'enhanced_agent_id' in globals() and enhanced_agent_id:
        bedrock.delete_agent(agentId=enhanced_agent_id, skipResourceInUseCheck=True)
        print(f"✅ Deleted Enhanced Bedrock Agent: {enhanced_agent_id}")
    else:
        print("⚠️ No enhanced_agent_id found")
except Exception as e:
    print(f"⚠️ Enhanced Agent cleanup error: {e}")

### Step 2: Delete Bridge Lambda

日本語訳:

ステップ 2: Bridge Lambda を削除する

次に、Bridge Lambda 関数を削除する必要があります。AWS Lambda コンソールを開き、関数の一覧から Bridge Lambda 関数を選択します。次に、「アクション」 > 「削除」を選択します。確認ダイアログボックスが表示されたら、「削除」をクリックして関数を削除します。

In [ ]:
lambda_client = boto3.client('lambda', region_name='us-east-1')

try:
    lambda_client.delete_function(FunctionName='enhanced-bedrock-gateway-bridge')
    print("✅ Deleted Bridge Lambda")
except Exception as e:
    print(f"⚠️ Bridge Lambda cleanup error: {e}")

### Step 3: Delete Enhanced Lambda Function

日本語訳:

### ステップ 3: 強化された Lambda 関数を削除する

次に、作成した強化された Lambda 関数を削除します。AWS Lambda コンソールで、関数の名前の横にあるチェックボックスをオンにし、「アクション」ドロップダウンメニューから「削除」を選択します。確認ダイアログボックスが表示されたら、「削除」をクリックして関数を削除します。

In [ ]:
lambda_client = boto3.client('lambda', region_name='us-east-1')

try:
    lambda_client.delete_function(FunctionName='enhanced-fruit-order-dynamo')
    print("✅ Deleted Enhanced Lambda")
except Exception as e:
    print(f"⚠️ Enhanced Lambda cleanup error: {e}")

### Step 4: Delete Gateway Target

日本語訳:

### ステップ 4: ゲートウェイターゲットの削除

1. AWS マネジメントコンソールにサインインし、 Amazon API Gateway コンソールを開きます。

2. 左側のナビゲーションペインで、「 REST API 」を選択します。

3. 対象の API を選択します。

4. 「リソース」を選択し、ターゲットリソースを選択します。

5. 「ターゲットの統合」セクションで、「ターゲットの編集」を選択します。

6. 「ターゲットの削除」を選択し、確認ダイアログボックスで「削除」をクリックして、ターゲットを削除します。

7. 変更を保存するには、「アクションの実行」を選択します。

ターゲットが正常に削除されたら、API Gateway コンソールに「正常に保存されました」というメッセージが表示されます。

In [ ]:
bedrock_agentcore_client = boto3.client(
    'bedrock-agentcore-control',
    region_name='us-east-1',
    endpoint_url="https://bedrock-agentcore-control.us-east-1.amazonaws.com"
)

try:
    if 'gateway_id' in globals() and gateway_id and 'target_id' in globals() and target_id:
        bedrock_agentcore_client.delete_gateway_target(
            gatewayIdentifier=gateway_id,
            targetId=target_id
        )
        print(f"✅ Deleted Gateway Target: {target_id}")
    else:
        print("⚠️ No gateway_id or target_id found")
except Exception as e:
    print(f"⚠️ Gateway Target cleanup error: {e}")

### Step 5: Delete Bedrock AgentCore Gateway

日本語訳:

### ステップ 5: Bedrock AgentCore Gateway を削除する

Bedrock AgentCore Gateway を削除するには、次の手順に従ってください。

1. Azure Portal で、リソースグループに移動します。
2. リソースグループから、削除する Bedrock AgentCore Gateway リソースを選択します。
3. 「概要」ブレードで、「削除」ボタンをクリックします。
4. 確認ダイアログボックスで、Bedrock AgentCore Gateway の名前を入力し、「OK」をクリックして削除を確認します。

削除プロセスが完了するまで数分かかる場合があります。削除が完了すると、リソースは Azure Portal のリソースグループから削除されます。

In [ ]:
try:
    if 'gateway_id' in globals() and gateway_id:
        bedrock_agentcore_client.delete_gateway(gatewayIdentifier=gateway_id)
        print(f"✅ Deleted Bedrock AgentCore Gateway: {gateway_id}")
    else:
        print("⚠️ No gateway_id found")
except Exception as e:
    print(f"⚠️ Gateway cleanup error: {e}")

### Step 6: DynamoDBテーブルの削除

日本語訳:
最後に、作成した DynamoDB テーブルを削除する必要があります。これを行うには、次の手順に従ってください。

1. AWS マネジメントコンソールにログインし、 DynamoDB コンソールを開きます。
2. 左側のナビゲーションペインで、「 Tables 」を選択します。
3. 削除するテーブルの横にあるチェックボックスをオンにします。
4. 上部にある「 Delete 」ボタンをクリックします。
5. テーブル名を入力するよう求められたら、削除するテーブル名を入力して「 Delete 」をクリックします。

これで、 DynamoDB テーブルが正常に削除されました。リソースを適切に削除することは、 AWS の利用コストを最小限に抑えるために重要です。

In [ ]:
dynamodb = boto3.client('dynamodb', region_name='us-east-1')

try:
    dynamodb.delete_table(TableName='FruitOrders')
    print("✅ Deleted FruitOrders table")
except Exception as e:
    print(f"⚠️ FruitOrders table cleanup error: {e}")

try:
    dynamodb.delete_table(TableName='FruitInventory')
    print("✅ Deleted FruitInventory table")
except Exception as e:
    print(f"⚠️ FruitInventory table cleanup error: {e}")

### Step 7: Cognito リソースの削除

日本語訳:
この手順では、Amazon Cognito で作成したリソースを削除します。

1. Amazon Cognito コンソールを開きます。
2. 「ユーザープール」を選択します。
3. 作成したユーザープールを選択し、「ユーザープール削除」をクリックします。
4. 「ユーザープール削除」ダイアログボックスで、ユーザープール名を入力して確認し、「ユーザープール削除」をクリックします。
5. 「ホストされたUI」を選択します。
6. 作成した hosted UI を選択し、「ホストされた UI 削除」をクリックします。
7. 「ホストされた UI 削除」ダイアログボックスで、hosted UI の名前を入力して確認し、「ホストされた UI 削除」をクリックします。

これで、Amazon Cognito のリソースが正常に削除されました。

In [ ]:
cognito = boto3.client('cognito-idp', region_name='us-east-1')

try:
    if 'auth_config' in globals() and auth_config:
        user_pool_id = auth_config.get('user_pool_id')
        domain_name = auth_config.get('domain_name')
        
        if user_pool_id and user_pool_id != 'us-east-1_0IalqBD2P':
            # Delete domain first

ドメインを最初に削除します。

To delete a domain, run the following gcloud command:

ドメインを削除するには、次の gcloud コマンドを実行します:

```
gcloud domains registrations delete [DOMAIN_NAME]
```

Replace [DOMAIN_NAME] with the domain you want to delete. You will be asked to confirm the deletion.

[DOMAIN_NAME] を削除したいドメインに置き換えます。削除の確認を求められます。

After deleting the domain, you can delete the Cloud DNS zone and any other associated resources.

ドメインを削除した後、Cloud DNS ゾーンや関連するその他のリソースを削除できます。
            try:
                cognito.delete_user_pool_domain(
                    Domain=domain_name,
                    UserPoolId=user_pool_id
                )
                print(f"✅ Deleted Cognito Domain: {domain_name}")
            except Exception as domain_error:
                print(f"⚠️ Domain deletion error: {domain_error}")
            
            # ユーザープールの削除

To delete a user pool, you can use the AWS Management Console, AWS CLI, or AWS SDKs. The general process is as follows:

1. Sign in to the AWS Management Console and navigate to the Amazon Cognito console.
2. Choose "Manage User Pools".
3. Select the user pool you want to delete.
4. Choose "Delete user pool".
5. Enter the user pool name to confirm the deletion.
6. Choose "Delete user pool" to complete the process.

Note that deleting a user pool is an irreversible action, and all data associated with the user pool, including users and groups, will be permanently deleted. Before deleting a user pool, make sure to back up any data you want to keep.

If you prefer to use the AWS CLI, you can run the following command to delete a user pool:

```
aws cognito-idp delete-user-pool --user-pool-id user_pool_id
```

Replace `user_pool_id` with the ID of the user pool you want to delete.

Alternatively, you can use one of the AWS SDKs to delete a user pool programmatically. The specific code will depend on the programming language you're using.

ユーザープールを削除するには、AWS Management Console、AWS CLI、または AWS SDK を使用できます。一般的な手順は次のとおりです。

1. AWS Management Console にサインインし、Amazon Cognito コンソールに移動します。
2. 「ユーザープールの管理」を選択します。
3. 削除するユーザープールを選択します。
4. 「ユーザープールの削除」を選択します。
5. ユーザープール名を入力して削除を確認します。
6. 「ユーザープールの削除」を選択してプロセスを完了します。

ユーザープールの削除は不可逆な操作であり、ユーザーやグループなど、ユーザープールに関連するすべてのデータが永久に削除されることに注意してください。ユーザープールを削除する前に、保持したいデータをバックアップしてください。

AWS CLI を使用する場合は、次のコマンドを実行してユーザープールを削除できます。

```
aws cognito-idp delete-user-pool --user-pool-id user_pool_id
```

`user_pool_id` は削除するユーザープールの ID に置き換えてください。

あるいは、AWS SDK のいずれかを使用して、プログラムでユーザープールを削除することもできます。具体的なコードは使用するプログラミング言語によって異なります。
            cognito.delete_user_pool(UserPoolId=user_pool_id)
            print(f"✅ Deleted Cognito User Pool: {user_pool_id}")
        else:
            print("⚠️ Using default user pool, skipping deletion")
    else:
        print("⚠️ No auth_config found")
except Exception as e:
    print(f"⚠️ Cognito cleanup error: {e}")

### Step 8: Delete IAM Role

日本語訳:
### ステップ 8: IAM ロールの削除

次のコマンドを使用して、作成した IAM ロール `lambda-ex` を削除します。

```
aws iam delete-role --role-name lambda-ex
```

`delete-role` コマンドは、指定したロール名に関連付けられているすべてのポリシーを削除し、ロールを削除します。ロールが削除されると、コンソールに `Role deleted` というメッセージが表示されます。

ロールを削除する前に、そのロールに関連付けられているリソース (Lambda 関数など) がないことを確認してください。関連リソースがある場合は、それらを削除するか、別のロールに関連付ける必要があります。

In [ ]:
iam = boto3.client('iam')

try:
    # Detach policies first

ポリシーを最初に分離してください。

To detach a policy from an IAM role, use the `detach-role-policy` command:

IAM ロールからポリシーを分離するには、`detach-role-policy` コマンドを使用します:

```
aws iam detach-role-policy --role-name my-role --policy-arn arn:aws:iam::123456789012:policy/UsefulPolicy
```

This command detaches the policy with the ARN `arn:aws:iam::123456789012:policy/UsefulPolicy` from the role named `my-role`.

このコマンドは、ARN `arn:aws:iam::123456789012:policy/UsefulPolicy` のポリシーを `my-role` という名前のロールから分離します。

To detach a policy from an IAM user, use the `detach-user-policy` command:

IAM ユーザーからポリシーを分離するには、`detach-user-policy` コマンドを使用します:

```
aws iam detach-user-policy --user-name my-user --policy-arn arn:aws:iam::123456789012:policy/UsefulPolicy
```

This command detaches the policy with the ARN `arn:aws:iam::123456789012:policy/UsefulPolicy` from the user named `my-user`.

このコマンドは、ARN `arn:aws:iam::123456789012:policy/UsefulPolicy` のポリシーを `my-user` という名前のユーザーから分離します。
    iam.detach_role_policy(
        RoleName='BedrockAgentCoreGatewayLambdaRole',
        PolicyArn='arn:aws:iam::aws:policy/AdministratorAccess'
    )
    print("✅ Detached AdministratorAccess policy")
    iam.delete_role_policy(
        RoleName='BedrockAgentCoreGatewayLambdaRole',
        PolicyName='BedrockAgentCoreInlinePolicy'
    )
    # Delete role

日本語訳:

# ロールの削除

この機能を使用すると、 `DELETE /roles/{role_id}` API エンドポイントを呼び出して、指定された `role_id` のロールを削除できます。ロールを削除すると、そのロールに関連付けられていたすべてのポリシーとユーザーの割り当ても削除されます。

**注意**: この操作を実行すると、元に戻すことはできません。ロールを削除する前に、そのロールに関連付けられているリソースを確認することをお勧めします。
    iam.delete_role(RoleName='BedrockAgentCoreGatewayLambdaRole')
    print("✅ Deleted IAM Role: BedrockAgentCoreGatewayLambdaRole")
    
except Exception as e:
    print(f"⚠️ IAM Role cleanup error: {e}")